# Import libraries and load data

In [ ]:
# import libraries

import numpy as np 
import pandas as pd 

# data viz libraries
import seaborn as sns
sns.set(style="whitegrid") # to make charts look better
import matplotlib.pyplot as plt
%matplotlib inline

# for functions
from tqdm import tqdm

# for ML
import datetime
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
pd.set_option('display.max_columns', None)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# import plotly modules
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.figure_factory as ff

# make it work on jupyter notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# use Plotly locally
cf.go_offline()

In [ ]:
# load datasets

dataFeatures = pd.read_csv("C:/Users/digit/Desktop/Ironhack/project-week-9-final-project/data/features.csv")
dataStores = pd.read_csv("C:/Users/digit/Desktop/Ironhack/project-week-9-final-project/data/stores.csv")
dataTest = pd.read_csv("C:/Users/digit/Desktop/Ironhack/project-week-9-final-project/data/test.csv")
dataTrain = pd.read_csv("C:/Users/digit/Desktop/Ironhack/project-week-9-final-project/data/train.csv")

# EDA and Data Cleaning

- Here we will explore the data in order to search for patterns, relationships and to understand the data better. 
- Perform data cleaning if neccessary and data wrangling.

In [ ]:
dataStores.head(5)

In [ ]:
dataStores.shape 

In [ ]:
dataFeatures.tail(5)

In [ ]:
dataFeatures.shape

In [ ]:
# we will start by merging dataStores and dataFeatures since Features is the extension of Stores
FeatSto = dataFeatures.merge(dataStores, how="inner", on="Store")

# check the head of the new df
FeatSto.head(5)

In [ ]:
FeatSto.shape

In [ ]:
# check the dtypes in FeatSto

FeatSto.dtypes

# Type is of categorical nature
# IsHoliday of binary categorical nature 

# the rest are numerical (Store and Size of discrete type, and the rest of continous type)
# some of the features might contain numerical values but still behave as categorical
# Date is string and we will convert it into datetime later or drop it

In [ ]:
# check for missing values
FeatSto.isnull().sum()

## Inspect the train and test data (dataTrain and dataTest)

In [ ]:
dataTest.head(5)

In [ ]:
dataTest.shape

In [ ]:
# as we can see  dataTrain includes additional Weekly_Sales
dataTrain.tail(5)

In [ ]:
dataTrain.shape

In [ ]:
dataTest.dtypes

In [ ]:
dataTrain.dtypes

In [ ]:
# we will disregard the dataTest and use only dataTrain
# (we will train-test split the data later)
# merge dataTrain with Featsto ->dfwTrain
# now we have a dataframe containing dataTrain
# FeatSto with dataTrain

dfwTrain = pd.merge(FeatSto, dataTrain, how="inner", on=["Store", "Date", "IsHoliday"])

dfwTrain.head(5)

In [ ]:
dfwTrain.shape

In [ ]:
dfwTrain.tail(5)

In [ ]:
# rename dfwTrain into df_total

df_total = dfwTrain

# show the head of the total dataframe

df_total.head()

In [ ]:
# show the tail

df_total.tail()

# first impression:
# Date is the week
# Markdowns 1 - 5 contain a lot of missing values
# Weekly_Sales is numerical continuous data

In [ ]:
# check the shape of df_total

df_total.shape

In [ ]:
# check the dtypes of df_total

df_total.dtypes

# most features have numerical values
# Date, Type is a string
# some features with numerical values might behave as categoricals, encode them later
# such as Type, IsHoliday

In [ ]:
# now we can check for missing values

df_total.isnull().sum()

In [ ]:
# calculate the percentage of missing values in each column

df_total.isnull().sum() / len(df_total)

# if the column contains 85% missing values then it should be dropped
# MarkDown1-5 contains anonymized data and lots of missing values, despite that, they contain important data

In [ ]:
# instead of dropping, we will fill the NaN values with zero values
# because the code was executed the first time, when we execute it again, it will show an error
df_total.fillna(0, inplace=True)

df_total.head()

In [ ]:
# check for missing values again

df_total.isnull().sum()

In [ ]:
# check for duplicated values

df_total.duplicated().sum()

# no duplicates

In [ ]:
# add a Month column

df_total["Month"] = pd.to_datetime(df_total['Date']).dt.month
df_total.sample(5) 

In [ ]:
# add a Week column 
df_total["Week"] = pd.to_datetime(df_total["Date"]).dt.week
df_total.sample(5)

In [ ]:
# add Year column
df_total["Year"] = pd.to_datetime(df_total["Date"]).dt.year 
df_total.sample(5)

In [ ]:
# convert "Date" column to datetime format
df_total["Date"] = pd.to_datetime(df_total["Date"])
df_total.dtypes

In [ ]:
df_total.head()

In [ ]:
# plot Average Monthly Sales - Per Year

weekly_sales_2010 = df_total[df_total.Year==2010]['Weekly_Sales'].groupby(df_total['Month']).mean()
weekly_sales_2011 = df_total[df_total.Year==2011]['Weekly_Sales'].groupby(df_total['Month']).mean()
weekly_sales_2012 = df_total[df_total.Year==2012]['Weekly_Sales'].groupby(df_total['Month']).mean()
plt.figure(figsize=(20,8))
sns.lineplot(weekly_sales_2010.index, weekly_sales_2010.values)
sns.lineplot(weekly_sales_2011.index, weekly_sales_2011.values)
sns.lineplot(weekly_sales_2012.index, weekly_sales_2012.values)
plt.grid()
plt.xticks(np.arange(1, 13, step=1))
plt.legend(['2010', '2011', '2012'], loc='best', fontsize=16)
plt.title('Average Monthly Sales - Per Year', fontsize=18)
plt.ylabel('Sales', fontsize=16)
plt.xlabel('Month', fontsize=16)
plt.show()

# 2012 compared to the rest was not doing so well

# there is a sharp rise in Sales between January and February, which is connected to SuperBowl
# and as we can see, the Monthly Sales are usually spiking in November and December
# when Thanksgiving and Christmas are happening



In [ ]:
# use Plotly to plot TimeSeries to see whether Date affects Weekly_Sales
# make one plot

px.line(df_total, x="Date", y="Weekly_Sales", labels={"x":"Date", "y":"Weekly_Sales"},
       title="Weekly Sales across Feb 2010 - Oct 2012")

# in more detail, we can see how Date affects Weekly Sales
# the highest spikes are on Thanksgiving Day and Christmas Day
# as we have seen in the previous plot, 2012 was not so good in terms of sales for Walmart

In [ ]:
df_total["Size"].value_counts()

In [ ]:
# use Plotly to plot a 3-dimensional lineplot
# we want to see whether Size has any impact on Weekly Sales
fig = px.line_3d(df_total, x='Year', y='Weekly_Sales', z='Size', color='Year')
fig

In [ ]:
# plot Average Weekly Sales per Store

weekly_sales = df_total["Weekly_Sales"].groupby(df_total["Store"]).mean()
fig = px.bar(weekly_sales, y="Weekly_Sales", labels={'Weekly_Sales':'Average Weekly Sales'}, title = "Average Weekly Sales per Store")
fig.show()

In [ ]:
# plot Average Weekly Sales per Department
weekly_sales = df_total["Weekly_Sales"].groupby(df_total["Dept"]).mean()
fig = px.bar(weekly_sales, y="Weekly_Sales", labels={'Weekly_Sales':'Average Weekly Sales'}, title = "Average Weekly Sales per Department")
fig.show()

In [ ]:
# now we can drop Date column now

df_total.drop(["Date"], inplace=True, axis=1)
df_total.sample(5)

#### Encode categorical features

For Linear Regression, we need to have numerical values. Thus we will encode the categorical features from the dataset into numerical values.

Since it's of categorical text data. We use Label Encoder to convert them into model-understandable numerical data.

In [ ]:
# encode Type

from sklearn.preprocessing import LabelEncoder
  
le = LabelEncoder()
df_total['Type']= le.fit_transform(df_total['Type'])

In [ ]:
# encode IsHoliday

df_total['IsHoliday'] = le.fit_transform(df_total['IsHoliday'])

In [ ]:
df_total.shape

In [ ]:
# took a sample with the size of 5000, which should be enough to better understand the relationship between the columns

# had problems with loading the plot, that's why I saved an image of it

# sns.pairplot(df_total.sample(5000), size = 5)

#from IPython.display import Image
#Image("sns_pairplot_df_total.png")


In [ ]:
# plot a scatter matrix in plotly (because sns.pairplot was too heavy)

fig = px.scatter_matrix(df_total.sample(1000), dimensions=['Store', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment',
       'IsHoliday', 'Type', 'Size', 'Dept', 'Weekly_Sales', 'Month', 'Week',
       'Year'], height=5000, width=5000, title="Scatter Matrix", size_max=20)
fig.show()

# no correlation between features mostly
# we can drop Type later
# we can drop Size later

In [ ]:
# check for correlations with correlation matrix
corr_matrix = df_total.corr(method="pearson") # we chose 'pearson'
corr_matrix

In [ ]:
# plot a heatmap for better understanding of correlation
fig, ax = plt.subplots(figsize=(14,12))
ax = sns.heatmap(corr_matrix, annot=True)
plt.show()

# values range between (-1,1)
# 0: no correlation at all
# 0 - 0.3: weak correlation
# 0.3 - 0.7: moderate correlation
# 0.7 - 1: strong correlation

# strong correlation between MarkDown1 and MarkDown4, drop MarkDown4 later
# Year and Fuel_Price show high correlation

Here we will further inspect the relationship between features and our target variable ("Weekly_Sales"), and features that are highly correlated between each other.

In [ ]:
# visualise a scatter plot in plotly 
# to see whether there is correlation between Unemployment and Weekly_Sales
# since Walmart is a huge retail company that's competitive thanks to cheap prices
# but I think Walmart is a default choice for a lot of people who do not know what they want
fig = px.scatter(df_total, x="Unemployment", y="Weekly_Sales")
fig.show()

Is there any correlation between the MarkDown1 -5 and Weekly_Sales?

In [ ]:
fig = px.scatter(df_total, x="MarkDown1", y="Weekly_Sales")
fig.show()

In [ ]:
fig = px.scatter(df_total, x="MarkDown2", y="Weekly_Sales")
fig.show()

In [ ]:
fig = px.scatter(df_total, x="MarkDown3", y="Weekly_Sales")
fig.show()

In [ ]:
fig = px.scatter(df_total, x="MarkDown4", y="Weekly_Sales")
fig.show()

In [ ]:
fig = px.scatter(df_total, x="MarkDown5", y="Weekly_Sales")
fig.show()

In [ ]:
fig = px.scatter(df_total, x="MarkDown1", y="MarkDown4")
fig.show()

# as we can see there is a positive correlation between MarkDown 1 and Markdown4

MarkDown 1-5 do not show strong correlation to Weekly_Sales. Fuel_Price shows strong correlation to Year. We will drop Fuel_Price otherwise they would carry similar information to the model. We won't drop Year as it differentiates the same Weeks for Store and Dept.

We can analyze other features that have weak with Weekly_Sales to see if they are useful.

In [ ]:
df_total.tail(5)

## Find outliers

Let's check for outliers in the features as LinearRegression is very sensitive to outliers.

(We could also see them from the scatter matrix that we plotted earlier)

In [ ]:
# plot a histogram to check frequency distribution and spot outliers

df_total.hist(figsize=(20,30), xrot=45, bins=50)
plt.show()

# Temperature is left-skewed (negative skewness)
# MarkDown1-5 heavily imbalanced - perform imputation and apply logarithmic transformation (if there is time)

In [ ]:
# plot histogram with plotly
#x1 = df_total["Store"]
#x2 = df_total["Temperature"]
#x3 = df_total["Fuel_Price"]

#x4 = df_total["MarkDown1"]
#x5 = df_total["MarkDown2"]
#x6 = df_total["MarkDown3"]
#x7 = df_total["MarkDown4"]
#x8 = df_total["MarkDown5"]

#x9 = df_total["CPI"]
#x10 = df_total["Unemployment"]
#x11 = df_total["IsHoliday"]
#x12 = df_total["Type"]
#x13 = df_total["Size"]
# x14 = df_total["Dept"]

# x15 = df_total["Weekly_Sales"]
# x16 = df_total["Month"]
# x17 = df_total["Week"]
# x18 = df_total["Year"]

# hist_data = [x1, x2, x3,x4, x5, x6, x7, x8, x9,x10, x11, x12,x13, x14, x15,x16, x17, x18]

# group_labels = ['Store', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment',
       'IsHoliday', 'Type', 'Size', 'Dept', 'Weekly_Sales', 'Month', 'Week',
       'Year']

# colors = ['#333F44', '#37AA9C', '#94F3E4', '#660000','#663300','#666600','#333300','#000000','#FF0000','#800000','#FFFF00',
          '#808000','#00FF00', '#00FFFF','#008080','#0000FF', '#000080', '#FF00FF']


# Create distplot with curve_type set to 'normal'
#fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)

# Add title
#fig.update_layout(title_text='Curve and Rug Plot')
#fig.show()

In [ ]:
fig = px.box(df_total, y="Temperature")
fig.show()

# Temperature has an outlier

In [ ]:
fig = px.box(df_total, y="Unemployment")
fig.show()

# Unemployment has outliers

In [ ]:
fig = px.box(df_total, y="Fuel_Price")
fig.show()

# Fuel Price has no outliers

# Linear Regression

## Objective

Choose the target variable y, and predict from features x. We want to fit a straight line to this data that minimizes the average squared distance between the data sample points and the fitted line. We can use the intercept and slope (which are coefficients) learned from this data to predict y (in this case Weekly Sales).

In [ ]:
# review data again to identify which label we want to predict

df_total.head()

In [ ]:
# first, we need to defined the target (dependent) variable we seek to predict
# we want to predict Weekly_Sales, isolate the y variable
y = df_total["Weekly_Sales"]

# then we drop the y variable from the features (X)
X = df_total.drop(["Weekly_Sales"], axis=1)

# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=47)
print(f"Length of train data: {len(X_train)}")
print(f"Length of test data: {len(X_test)}")

In [ ]:
X_train.shape, y_train.shape

In [ ]:
# import the model
from sklearn import linear_model

# import evaluation metrics
from sklearn.metrics import mean_squared_error, r2_score

# define the regression model
lm = linear_model.LinearRegression()

In [ ]:
# next, we fit the model to our data
lm.fit(X_train, y_train)

# then calculate a score

lm.score(X_train,y_train) # the coefficient of determination R2 

In [ ]:
r2_score(y_train, y_pred)

## Interpret the Coefficients

The coefficients(b0 and b1) will allow us to model our equation with values and find the best fit line. The linear_regressor variable (assigned to a LinearRegression object), is able to have the intercept and coefficients extracted, using the code below.

In [ ]:
# prints y-intercept
print(lm.intercept_)

# prints the coefficient
print(lm.coef_)

## Making predictions based on our model

Now that we have trained our algorithm, it’s time to make some predictions. To do so, we will use our test data and see how accurately our algorithm predicts the Weekly Sales.

Making predictions based on our model, we will use the code below to pass the predict method to our test data. This will return predicted values of target y given the new test X data.

In [ ]:
# now we have the first imperfect iteration (It1)
# y_pred is prediction of y

y_pred= lm.predict(X_test) # make predictions
y_pred

In [ ]:
# check the shape of the training and testing data
print(X_test.shape, y_test.shape, X_train.shape, y_train.shape, y_pred.shape)

## Model Evaluation

There are three primary metrics used to evaluate linear models. These are: Mean absolute error (MAE), Mean squared error (MSE), or Root mean squared error (RMSE).

In [ ]:
# import metrics library
from sklearn import metrics

# print result of MAE
print(metrics.mean_absolute_error(y_test, y_pred))

#print result of MSE
print(metrics.mean_squared_error(y_test, y_pred))

#print result of RMSE
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

## Conclusion

The model is not performing well, because we ran a Simple Linear Regression, which operates in 2 dimensions. The dataset is too large and has multiple dimensions.

So we will try to run Multiple Linear Regression.

# Multiple Linear Regression

## Objective

We want to predict Weekly Sales based on the features.

In [ ]:
# make a copy of a df_total

df2_total = df_total.copy(deep=True)

In [ ]:
# we will drop MarkDown 4 and Fuel_Price 
df2_total = df2_total.drop(["MarkDown4", "Fuel_Price"], axis=1)
df2_total.columns

## Normalize Data For Comparison

We need to scale the data to the range of 0 to 1. We will use MinMaxScaler()

In [ ]:
from sklearn import preprocessing

# Scale and plot the features against Weekly_Sales (target) using the MinMax scaler (Normalization)
min_max_scaler = preprocessing.MinMaxScaler()
col_name = df2_total.drop('Weekly_Sales', axis = 1).columns[:]
x = df2_total.loc[:, col_name]
y = df2_total['Weekly_Sales']

# Normalizing x
x = pd.DataFrame(data = min_max_scaler.fit_transform(x), columns = col_name)

# Examine the normalized data
print(df2_total.head())
x.head()

In [ ]:
# run df_total for comparison
df_total.head()

Now, we would like to examine the relationship of each feature against the target (price). We can do this through sns.regplot(). regplot() will also try to draw a best fit line to show the linear relationship between each feature and the target.

In [ ]:
# plot heatmap to visualize the data after normalisation

plt.figure(figsize = (20, 10))
sns.heatmap(df2_total.corr(), annot = True)
plt.show()

Now we will split the dataset into training set and testing set.
We usually use 60 -80 % for training and 20 - 40 % for testing.

X is the input dataset to the model
y is the output dataset to the model
test_size: the percent of data that we want to use for testing, usually from (0.2 - 0.4)
random_state: randomly split the train and test dataset

In [ ]:
# drop the y variable from the features (X)
X = df2_total.drop('Weekly_Sales', axis = 1)
# we want to predict Weekly_Sales, isolate the y variable 
y = df2_total['Weekly_Sales']

# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=47)
print("Train features shape : ", X_train.shape)
print("Train target shape   : ", y_train.shape)
print("Test features shape  : ", X_test.shape)
print("Test target shape    : ", y_test.shape)

## Model Building 

In [ ]:
# build the model
model = LinearRegression(normalize = True) # the parameter normalized = True enables the data to be normalized when fed into the model
# fit the training data into the model
model.fit(X_train, y_train)

## Interpret The Model

We generated a LinearRegression model that consist of coefficients and intercept. We can now have a look at the intercept and coefficients for our model and interpret them.

In [ ]:
print("Model intercept  : ", model.intercept_, "\n")
print("Model coefficient: ", model.coef_, "\n")

for i in range(len(X.columns)):
    print(X.columns[i], ": ", model.coef_[i])

In [ ]:
# Model evaluation for training set
y_train_pred = model.predict(X_train)
rmse = (np.sqrt(mean_squared_error(y_train, y_train_pred)))
r2 = r2_score(y_train, y_train_pred)

# Examine the first 10 predicted output from the model
output = pd.DataFrame(y_train[0:15])
output['Predicted'] = y_train_pred[0:15]
output['Difference'] = output['Predicted'] - output['Weekly_Sales']
print(output, "\n")

print("Model training performance:")
print("---------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

# Model evaluation for testing set
y_test_pred = model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, y_test_pred)))
r2 = r2_score(y_test, y_test_pred)

output = pd.DataFrame(y_test[0:15])
output['Predicted'] = y_test_pred[0:15]
output['Difference'] = output['Predicted'] - output['Weekly_Sales']
print(output, "\n")

print("Model testing performance:")
print("--------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

The r2 score of our model is just 0.069 and the difference between the actual and predicted value is high.

Why is this happening?

- The features have no linear relationship with Weekly_Sales.
- Features need to be further cleaned and engineered.

How to solve it?

- Clean the outliers and invalid data.
- Try out other models such as DecisionTreeRegressor and GradientBoostingRegressor.

- These models can be found in the scikit-learn documentation. With that, we need to examine their r2 score, MSE, RMSE and MAE and compare it with LinearRegression model.